# 线性判别分析（LDA）
### 1.用途：数据预处理中的降维，分类任务
### 2.目标：LDA关心的是能够最大化类间区分度的坐标轴成分
将特征空间（数据集中的多维样本）投影到一个维度更小的 k 维子空间中，
同时保持区分类别的信息
#### LDA是“有监督”的，它计算的是另一类特定的方向
#### 投影的目的：找到更合适分类的空间
#### 与PCA不同，更关心分类而不是方差

## 3.理论：
二分类举例：假设有数据集x，投影函数为 $ y = w^Tx $
#### LDA分类的一个目标是使得不同类别之间的距离越远越好，同一类别之中的距离越近越好。
1.计算每类样本的均值：$ u_i = \frac{1}{N_i}\sum_{x∈D_i}x $ <br>
2.使用映射函数将原样本映射到新的样本空间中：$ y = w^Tx $ <br>
3.计算投影后的均值：$ \overline{u_i} = \frac{1}{N_i}\sum_{y∈D_i}y = \frac{1}{N_i}\sum_{x∈D_i}w^Tx = w^Tu_i $ <br/>
4.使得投影后两类样本中心点尽量分离，即：$$ J(w) = |\overline{u_1} - \overline{n_2}| = |w^T(u_1-u_2)| $$
** 那么只需要最大化J(w)就可以了吗？ <font color='red'>不是，若类内的离散度很高，虽然两类中心相聚较远，但是仍然会存在映射后两类样本重叠的情况</font> ** <br/>
所以，仅仅使用类间中心距离是不够的，还需要类内的离散度来保证 <br/>
5.同类之间应该密集一些，即：$$ \overline{s_i^2} = \sum_{y∈D_i}(y - \overline{u_i})^2 $$ <br/>
6.那么目标函数就转换为：$$ J(w) = \frac{|\overline{u_1} - \overline{u_2}|^2}{\overline{s_1}^2 + \overline{s_2}^2} $$
7.将散列值公式展开：$$ \overline{s_i^2} = \sum_{y∈D_i}(y - \overline{u_i})^2 = \sum_{x∈D_i}(w^Tx-w^Tu_i)^2 = \sum_{x∈D_i}{w^T(x-u_i)(x-u_i)^Tw} $$
其中散列矩阵为：$ s_i = \sum_{x∈D_i}(x-u_i)(x-u_i)^T $ <br>
8.类内的散列矩阵$S_w = S_1+S_2$
9.计算类间散列矩阵：$$(\overline{u_1} - \overline{u_2})^2 = (w^Tu_1-w^Tu2)^2 = w^T(u_1-u_2)(u_1-u_2)^Tw = w^TS_Bw$$
其中$S_B$称为类间散列矩阵 <br>
10.所以，6中的目标转换为$J(w) = \frac{w^TS_Bw}{w^TS_ww}$
11.分母进行归一化：如果分子、分母是都可以取任意值的，那就会使得有无穷解，我们将分母限制为长度为1。然后利用拉个朗日乘子法求解：$$ c(w) = w^TS_Bw-λ(w^TS_ww-1) $$ $$ ==>\frac{dc}{dw} = 2S_Bw - 2λS_ww 另其为 0 $$ $$ ==> S_Bw = λS_ww $$
如果$S_w$是非奇异矩阵时，两边t乘以$ S_w $的逆矩阵，即：$$ S_w^{-1}S_Bw = λw $$（w就是矩阵$S_w^{-1}S_B$的特征向量，只要解出特征向量即可）

In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [15]:
feature_dict = {i:label for i,label in zip(
                range(4),
                ('sepal length in cm',
                 'sepal width in cm',
                 'pepal length in cm',
                 'petal width in cm'))}

In [18]:
df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",header=None)
df.columns = [l for i,l in sorted(feature_dict.items())] + ['class label']
df.tail()

,sepal length in cm,sepal width in cm,pepal length in cm,petal width in cm,class label
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


** X ** = $\left[ \begin{array} {X}
x1_{sepal\_length} & x1_{sepal\_width} & x1_{petal\_length} & x1_{petal\_width} \\
x2_{sepal\_length} & x2_{sepal\_width} & x2_{petal\_length} & x2_{petal\_width} \\
\cdots & \cdots & \cdots & \cdots \\
x150_{sepal\_length} & x150_{sepal\_width} & x150_{petal\_length} & x150_{petal\_width} \\
\end{array} \right] $ ，**y** = $\left[ \begin{array} {y}
w_{setosa} \\
w_{setosa} \\
\cdots \\
w_{virginica} \\
\end{array} \right] $

In [24]:
X = df[['sepal length in cm','sepal width in cm','pepal length in cm','petal width in cm']].values
y = df['class label'].values

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y)+1

In [26]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

**y** = $\left[ \begin{array} {y}
setosa \\
setosa \\
\cdots \\
virginica \\
\end{array} \right] = 
\left[ \begin{array} {y}
1 \\
1 \\
\cdots \\
3 \\
\end{array} \right]
$
### 分别求三种鸢尾花数据在不同维度上的均值向量$m_i$
$m_i = \left[ \begin{array} {y}
u_{wi(sepal\_length)} \\
u_{wi(sepal\_width)} \\
u_{wi(petal\_length)} \\
u_{wi(petal\_length)} \\
\end{array} \right]
， with $ i = 1,2,3  

In [28]:
np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,4):
    mean_vectors.append(np.mean(X[y==cl],axis=0))
    print("Mean Vector class %s：%s\n"%(cl,mean_vectors[cl-1]))

Mean Vector class 1：[5.006 3.418 1.464 0.244]

Mean Vector class 2：[5.936 2.77  4.26  1.326]

Mean Vector class 3：[6.588 2.974 5.552 2.026]



## 计算两个4X4维矩阵：类内散步矩阵和类间散步矩阵
### 类内散布矩阵：$$ S_W = \sum_{i=1}^{c}S_i $$ $$ S_i = \sum_{x∈D_i}(x - m_i)(x-m_i)^T $$ $$ m_i = \frac{1}{n_i}\sum_{x∈D_i}x_k  $$

In [29]:
S_W = np.zeros((4,4))
for cl,mv in zip(range(1,4),mean_vectors):
    class_sc_mat = np.zeros((4,4)) # 每一个类的类内散步矩阵
    for row in X[y == cl]:
        row,mv = row.reshape(4,1),mv.reshape(4,1)
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat
print("within-class Scatter Matrix:\n",S_W)

within-class Scatter Matrix:
 [[38.9562 13.683  24.614   5.6556]
 [13.683  17.035   8.12    4.9132]
 [24.614   8.12   27.22    6.2536]
 [ 5.6556  4.9132  6.2536  6.1756]]


## 类间散步矩阵：$$ S_B = \sum_{i=1}^{c}N_i(m_i-m)(m_i-m)^T $$
m是全局均值，而$m_i$和$N_i$是每类样本的均值和样本数

In [30]:
overall_mean = np.mean(X,axis=0)
S_B = np.zeros((4,4))
for i,mean_vec in enumerate(mean_vectors):
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(4,1)
    overall_mean = overall_mean.reshape(4,1)
    S_B += n*(mean_vec-overall_mean).dot((mean_vec-overall_mean).T)
print("between-class Scatter Matrix:\n",S_B)

between-class Scatter Matrix:
 [[ 63.2121 -19.534  165.1647  71.3631]
 [-19.534   10.9776 -56.0552 -22.4924]
 [165.1647 -56.0552 436.6437 186.9081]
 [ 71.3631 -22.4924 186.9081  80.6041]]


# $$ 求解矩阵的特征值：S_w^{-1}S_B $$

In [33]:
eig_vals,eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
print("特征值：\n",eig_vals)
print("特征向量：\n",eig_vecs)

特征值：
 [3.2272e+01 2.7757e-01 3.4225e-15 1.1483e-14]
特征向量：
 [[-0.2049 -0.009  -0.8844 -0.2234]
 [-0.3871 -0.589   0.2854 -0.2523]
 [ 0.5465  0.2543  0.258  -0.326 ]
 [ 0.7138 -0.767   0.2643  0.8833]]


## 特征值和特征向量：
* 特征向量：表示映射方向
* 特征值：特征向量的重要性程度

In [39]:
eig_pairs = [(np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]
#排序
eig_pairs = sorted(eig_pairs,key=lambda k:k[0],reverse=True)
print("特征值递减排序：\n")
for i in eig_pairs:
    print(i[0])

特征值递减排序：

32.27195779972981
0.27756686384003953
1.1483362279322388e-14
3.422458920849769e-15


In [ ]:
eigv_sum = sum(eig_vals)
print("特征重要性：")
for i,j in enumerate(eig_pairs):
    print("eigenvalue {0:}: {1:.2%}".format(i+1,(j[0]/eigv_sum)))

## 选择前两维特征

In [43]:
W = np.hstack((eig_pairs[0][1].reshape(4,1),eig_pairs[1][1].reshape(4,1)))
print("Matrix W:\n",W)

Matrix W:
 [[-0.2049 -0.009 ]
 [-0.3871 -0.589 ]
 [ 0.5465  0.2543]
 [ 0.7138 -0.767 ]]


In [47]:
#降维
X_1da = X.dot(W)
assert X_1da.shape == (150,2),"The matrix is not 150x2 demensional."